In [ ]:
import argparse
import logging
import snappy
import apache_beam as beam
import apache_beam.transforms.window as window
from apache_beam.examples.wordcount import WordExtractingDoFn
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.transforms.window import TimestampCombiner
from apache_beam.transforms.core import CombinePerKey, GroupByKey
from apache_beam.io.avroio import ReadFromAvro
from apache_beam.transforms.combiners import Sample
import json
import faust
from operator import itemgetter
import numpy as np
from apache_beam.transforms.userstate import BagStateSpec, CombiningValueStateSpec,  TimerSpec, on_timer, StateSpec
from apache_beam.coders.coders import VarIntCoder, PickleCoder, BytesCoder, StrUtf8Coder, FastPrimitivesCoder
from apache_beam.transforms.timeutil import TimeDomain
from apache_beam.transforms.combiners import CountCombineFn
import time
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.util import assert_that
from apache_beam.testing.util import equal_to
import unittest
from apache_beam.metrics.metric import Metrics
from apache_beam.runners.direct.direct_runner import DirectRunner
from apache_beam.transforms import core
from apache_beam.transforms import cy_combiners
from apache_beam.transforms import ptransform
from apache_beam.transforms import window
from apache_beam.transforms.display import DisplayDataItem
from apache_beam.typehints import KV
from apache_beam.typehints import Any
from apache_beam.typehints import Dict
from apache_beam.typehints import Iterable
from apache_beam.typehints import List
from apache_beam.typehints import Tuple
from apache_beam.typehints import TypeVariable
from apache_beam.typehints import Union
from apache_beam.typehints import with_input_types
from apache_beam.typehints import with_output_types
import faust
import json
from datetime import datetime

In [ ]:
t = Trade(
    event_id=["binance", "trade", "BTC", "USDT", "1557668999913"],
    event_time=1557669000,
    event_type="trade",
    exchange="binance",
    symbol="BTCUSDT",
    quote_asset="USDT",
    base_asset="BTC",
    quantity=0.024059,
    price=7134.78
)

In [ ]:
trade = t.dumps()

In [ ]:
Trade.loads('{"event_id": ["binance", "trade", "BTC", "USDT", "1557668999913"], "event_time": 1557669000, "event_datetime": null, "event_type": "trade", "event_time_ms": null, "produced_time": null, "produced_datetime": null, "exchange": "binance", "symbol": "BTCUSDT", "quote_asset": "USDT", "base_asset": "BTC", "market_type": "spot", "trade_time": null, "trade_id": null, "quantity": 0.024059, "price": 7134.78, "side": null, "is_maker": null, "order_type": null}')

In [ ]:
class ParseTrade(beam.DoFn):
   def process(self, element):
        data = json.loads(element["message"].decode("utf-8"))
        del data["__faust"]
        trade = Trade.loads(json.dumps(data))
        return [trade]
    
class ParseDepth(beam.DoFn):
   def process(self, element):
        data = json.loads(element["message"].decode("utf-8"))
        del data["__faust"]
        trade = Depth.loads(json.dumps(data))
        return [trade]

class PrintDoFn(beam.DoFn):
   def process(self, element):
        print(element)
        return [element]


In [ ]:
with TestPipeline() as p:
    tradeRecords = (p 
        | 'ReadTrades' >> ReadFromAvro('../data/depths_avro/depths.avro',use_fastavro=True)
        | 'FormatTradesPubsub' >> beam.ParDo(ParseDepth())
        | 'PrintTrades' >> beam.Map(lambda x: print(x))
    )

In [ ]:
with TestPipeline() as p:
    tradeRecords = (p 
        | 'ReadTrades' >> ReadFromAvro('../data/depths_avro/depths.avro',use_fastavro=True)
        | 'FormatDepthsPubsub' >> beam.ParDo(ParseDepth())
        | 'PrintDepths' >> beam.Map(lambda x: print(x))
    )

In [ ]:

with TestPipeline() as p:
    tradeRecords = (p 
    | 'ReadTrades' >> ReadFromAvro('./trades.avro',use_fastavro=True)
    | 'FormatTradesPubsub' >> beam.ParDo(FormatPubsubDoFn())
    )

    # ==================================================================>
    # DepthUpdaate from Avro
    # ==================================================================>

    depthUpdateRecords = (p 
        | 'ReadDepthUpdates' >> ReadFromAvro('./depths.avro',use_fastavro=True)
        | 'FormatDepthUpdatePubsub' >> beam.ParDo(FormatPubsubDoFn())
    )
    
    mergedRecords = ((depthUpdateRecords,tradeRecords) 
        | 'MergePCollections' >> beam.Flatten()
        | 'AddEventTimestamp' >> beam.Map(lambda e: beam.window.TimestampedValue(e, e["event_time_ms"]))
    )
    
    (mergedRecords 
        | 'FilterTrades' >> beam.Filter(lambda x: x["event_type"] == "trade")
    )
    
    (mergedRecords 
        | 'FilterDepthUpdates' >> beam.Filter(lambda x: x["event_type"] == "depthUpdate")
    )